# Text summarization with ALBERT

## Introduction

## 1.0 Setup

Let's check out what kind of GPU our friends at Google gave us.

In [0]:
!nvidia-smi

First, we clone the Hugging Face transformer library from Github.


Note it's checking out a specific commit only because I've tested this

In [0]:
!git clone https://github.com/google-research/albert.git
!pip install -r albert/requirements.txt

In [0]:
!pip install tensorboardX
!pip install tensorflow-datasets

## 2.0 Train Model


### 2.1 Get Training and Evaluation Data
get CNN Daily Mail Datatest from the tensorflow dataset library
https://www.tensorflow.org/datasets


In [0]:
import tensorflow_datasets as tfds
cnn_dailymail = tfds.load(name="cnn_dailymail", data_dir="dataset")

In [0]:
cnn_dailymail['train'].size

In [0]:
# https://github.com/google-research/text-to-text-transfer-transformer/blob/master/t5/evaluation/metrics.py

def rouge(targets, predictions, score_keys=None):
  """Computes rouge score.
  Args:
    targets: list of strings
    predictions: list of strings
    score_keys: list of strings with the keys to compute.
  Returns:
    dict with score_key: rouge score across all targets and predictions
  """

  if score_keys is None:
    score_keys = ["rouge1", "rouge2", "rougeLsum"]
  scorer = rouge_scorer.RougeScorer(score_keys)
  aggregator = scoring.BootstrapAggregator()

  def _prepare_summary(summary):
    # Make sure the summary is not bytes-type
    # Add newlines between sentences so that rougeLsum is computed correctly.
    summary = summary.replace(" . ", " .\n")
    return summary

  for prediction, target in zip(predictions, targets):
    target = _prepare_summary(target)
    prediction = _prepare_summary(prediction)
    aggregator.add_scores(scorer.score(target=target, prediction=prediction))
  result = aggregator.aggregate()
  for key in score_keys:
    logging.info(
        "%s = %.2f, 95%% confidence [%.2f, %.2f]",
        key,
        result[key].mid.fmeasure*100,
        result[key].low.fmeasure*100,
        result[key].high.fmeasure*100,
    )
  return {key: result[key].mid.fmeasure*100 for key in score_keys}

In [0]:
ALBERT_MODEL = 'base' #@param {type:"string"}
OUTPUT_DIR = 'out_albert' #@param {type:"string"}
ALBERT_MODEL_HUB = 'https://tfhub.dev/google/albert_' + ALBERT_MODEL + '/3'

### 2.2 Run training 

We can now train the model with the training set. 



In [0]:
!python -m albert.run_classifier \
  --data_dir="dataset/cnn_dailymail" \
  --output_dir=$OUTPUT_DIR \
  --albert_hub_module_handle=$ALBERT_MODEL_HUB \
  --spm_model_file="from_tf_hub" \
  --do_train \
  --do_eval \
  --do_predict \
  --do_lower_case \
  --max_seq_length=512 \
  --optimizer=adamw \
  --task_name=MNLI \
  --warmup_step=1000 \
  --learning_rate=3e-5 \
  --train_step=10000 \
  --save_checkpoints_steps=100 \
  --train_batch_size=32